In [1]:
import numpy as np
import pandas as pd
import cv2
import os
from tqdm import tqdm # 로컬에서는 tqdm.notebook 대신 일반 tqdm 사용
from tqdm.contrib.concurrent import thread_map
import joblib
import random

# skimage
from skimage.exposure import rescale_intensity, equalize_hist
from skimage.filters import gaussian
from skimage.restoration import denoise_bilateral
from skimage.util import img_as_ubyte
from skimage.feature import local_binary_pattern, hog, graycomatrix, graycoprops
from skimage.transform import resize
from scipy import ndimage

# sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder,normalize
from sklearn.metrics import pairwise_distances, silhouette_score, confusion_matrix
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import resample


# joblib (병렬 처리를 위해)
from joblib import Parallel, delayed

# faiss (KNN 가속화를 위해)
import faiss

# 데이터 관련 
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

import seaborn as sns
import matplotlib.pyplot as plt

c:\Users\bvb09\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def edge(img_bgr):
    # 1) 강제 리사이즈
    img_bgr = cv2.resize(img_bgr, (120, 120), interpolation=cv2.INTER_AREA)

    # 2) CLAHE → Gray → Blur → Denoise → Canny
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(2.0, (8,8))
    l = clahe.apply(l)
    img_eq = cv2.cvtColor(cv2.merge((l,a,b)), cv2.COLOR_LAB2BGR)

    gray = cv2.cvtColor(img_eq, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3,3), 0)
    denoised = cv2.fastNlMeansDenoising(blurred, h=10, templateWindowSize=7, searchWindowSize=21)
    edges = cv2.Canny(denoised, 100, 230)   

    return edges

# 1) 공통: BGR → LAB → CLAHE(L) → BGR (필요 시)
def apply_clahe(img_bgr):
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = cv2.createCLAHE(2.0, (8,8)).apply(l)
    return cv2.cvtColor(cv2.merge((l,a,b)), cv2.COLOR_LAB2BGR)

# 3) Unsharp mask (SIFT 전용)
def unsharp(img_gray):
    blurred = cv2.GaussianBlur(img_gray, (3,3), 0)
    return cv2.addWeighted(img_gray, 1.5, blurred, -0.5, 0)

# 4) Mild Gaussian blur (LBP/GLCM/Laws)
def mild_blur(img_gray):
    return cv2.GaussianBlur(img_gray, (3,3), 0)
def gray(img):
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return image

# ==============================================================================
#  피쳐 추출 함수
# ==================================================================:============

def extract_color_histogram_features(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0,1,2], None, (8,8,8), [0,180,0,256,0,256])
    return cv2.normalize(hist, hist).flatten()

def extract_sift_pca_mean(img_bgr, pca_model=None, n_components=32):
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img_gray, None)

    if descriptors is None or len(descriptors) == 0:
        return np.zeros(n_components if pca_model else 128, dtype=np.float32)

    if pca_model is None:
        pca_model = PCA(n_components=n_components)
        descriptors_pca = pca_model.fit_transform(descriptors)
    else:
        descriptors_pca = pca_model.transform(descriptors)

    mean_vector = np.mean(descriptors_pca, axis=0)
    return mean_vector.astype(np.float32)

def extract_glcm_features(image):
    if image is None:
        num_props = 6
        num_distances = 3
        num_angles = 4
        return np.zeros(num_props * num_distances * num_angles)
        
    img_glcm = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_glcm = mild_blur(img_glcm)
    img_glcm = apply_clahe(cv2.cvtColor(img_glcm, cv2.COLOR_GRAY2BGR))
    gray_image = cv2.cvtColor(img_glcm, cv2.COLOR_BGR2GRAY)

    distances = [1, 2, 3]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    
    try:
        glcm = graycomatrix(gray_image, distances=distances, angles=angles, symmetric=True, normed=True)
        
        props_to_extract = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
        glcm_features = []
        for prop in props_to_extract:
            glcm_features.append(graycoprops(glcm, prop).ravel())
            
        return np.concatenate(glcm_features)
    except Exception as e:
        print(f"GLCM 추출 중 오류 발생: {e}")
        num_props = 6
        num_distances = len(distances) 
        num_angles = len(angles)   
        return np.zeros(num_props * num_distances * num_angles)

def extract_hog_features(image, orientations=9, pixels_per_cell=(16, 16), cells_per_block=(2, 2)):
    img = cv2.resize(image, (120, 120), interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    features = hog(gray,
                   orientations=orientations,
                   pixels_per_cell=pixels_per_cell,
                   cells_per_block=cells_per_block,
                   block_norm='L2-Hys',
                   visualize=False,
                   transform_sqrt=True,
                   feature_vector=True)
    return features.astype(np.float32)

def extract_sift_descriptors_from_array(image):
    img = apply_clahe(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_image=unsharp(img)

    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(gray_image, None)

    return des

def extract_lbp_features_from_array(image, P=8, R=1, method='uniform'):
    image=apply_clahe(image)
    image=mild_blur(image)
    gray_image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp_image = local_binary_pattern(gray_image, P, R, method=method)

    max_bins = P * (P - 1) + 3 if method == 'default' else P + 2
    hist, _ = np.histogram(lbp_image.ravel(), bins=max_bins, range=(0, max_bins), density=True)
    return hist


# Laws' Texture Energy - 기존과 동일
def extract_laws_energy_features(image, window_size=15):
    image_gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_gray=mild_blur(image_gray)

    L5 = np.array([1, 4, 6, 4, 1], dtype=np.float32)
    E5 = np.array([-1, -2, 0, 2, 1], dtype=np.float32)
    S5 = np.array([-1, 0, 2, 0, -1], dtype=np.float32)
    W5 = np.array([-1, 2, 0, -2, 1], dtype=np.float32)
    R5 = np.array([1, -4, 6, -4, 1], dtype=np.float32)
    kernels = [L5, E5, S5, W5, R5]

    energy_features = []
    if image_gray.dtype == np.uint8:
        image_gray = image_gray.astype(np.float32)

    for k1 in kernels:
        for k2 in kernels:
            kernel = np.outer(k1, k2)
            filtered = ndimage.convolve(image_gray, kernel, mode='reflect')
            energy = np.abs(filtered)
            summed = cv2.boxFilter(energy, ddepth=-1, ksize=(window_size, window_size), normalize=False)
            energy_features.append(summed.mean())

    return np.array(energy_features, dtype=np.float32)


def learn_bovw_vocabulary(all_sift_descriptors, num_clusters=200):
    filtered = [des for des in all_sift_descriptors if des is not None and len(des) > 0]
    feature_dims = {des.shape[1] for des in filtered}
    if len(feature_dims) > 1:
        raise ValueError(f"❌ BoVW 학습용 SIFT 디스크립터들의 차원이 일치하지 않음: {feature_dims}")

    concatenated_descriptors = np.vstack(filtered)
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS
    _, _, centers = cv2.kmeans(
        concatenated_descriptors.astype(np.float32),
        num_clusters, None, criteria, 10, flags
    )
    return centers

def create_bovw_histogram(sift_descriptors, vocabulary):
    num_clusters = vocabulary.shape[0]
    if sift_descriptors is None or len(sift_descriptors) == 0:
        return np.zeros(num_clusters, dtype=np.float32)

    try:
        distances = np.linalg.norm(
            vocabulary[None, :, :] - sift_descriptors[:, None, :], axis=2
        )
        closest_clusters = np.argmin(distances, axis=1)
        histogram = np.bincount(closest_clusters, minlength=num_clusters).astype(np.float32)
        histogram = cv2.normalize(histogram, None, norm_type=cv2.NORM_L2).flatten()
        return histogram

    except Exception as e:
        print(f"⚠️ BoVW 히스토그램 생성 오류: {e}")
        return np.zeros(num_clusters, dtype=np.float32)


def parallel_create_bovw_histograms(descriptor_list, vocabulary, n_jobs=6):
    histograms = Parallel(n_jobs=n_jobs)(
        delayed(create_bovw_histogram)(desc, vocabulary) for desc in descriptor_list
    )
    return np.array(histograms, dtype=np.float32)

print("✔ 특징 추출 함수 정의 완료")

# ==============================================================================
#  분류 모델 및 학습/평가 함수
# ==============================================================================
# 유클리드 거리 기반 Faiss KNN 학습 (수정됨)

def combine_features(*feature_arrays):
    """주어진 특징 배열들을 가로로 결합합니다."""
    return np.hstack(feature_arrays)

def train_faiss_knn_euclidean(X_train, y_train, n_neighbors=3):
    X_train = X_train.astype(np.float32)
    D = X_train.shape[1]

    index = faiss.IndexFlatL2(D)
    index.add(X_train)

    return index, y_train, n_neighbors

# 유클리드 거리 기반 Faiss KNN 예측 (수정됨)
def predict_faiss_knn_euclidean(index, y_train_labels, n_neighbors, X_test):
    X_test = X_test.astype(np.float32)

    distances, indices = index.search(X_test, n_neighbors)

    y_pred = []
    for i in range(len(X_test)):
        neighbor_labels = y_train_labels[indices[i]]
        unique_labels, counts = np.unique(neighbor_labels, return_counts=True)
        predicted_label = unique_labels[np.argmax(counts)]
        y_pred.append(predicted_label)
    return np.array(y_pred)

# 기존 predict_faiss_knn_euclidean 함수를 수정하거나, 아래 함수를 추가합니다.
# 이 함수는 k개의 가장 가까운 이웃의 라벨을 반환합니다.
def predict_faiss_knn_topk(faiss_index, train_labels_encoded, query_features, k=10):
    if faiss_index is None:
        raise ValueError("Faiss 인덱스가 학습되지 않았습니다.")
    if query_features.shape[0] == 0:
        return np.array([]), np.array([]), np.array([])

    D, I = faiss_index.search(query_features, k)
    predicted_neighbor_labels = train_labels_encoded[I]

    return D, I, predicted_neighbor_labels

# 참고: 기존 predict_faiss_knn_euclidean 함수는 단순히 k=1로 설정하고 첫 번째 라벨을 반환하는 식으로 동작할 수 있습니다.
# 만약 기존 predict_faiss_knn_euclidean 함수가 k를 인자로 받는다면, 그 함수를 활용해도 좋습니다.
# 여기서는 Top-k 결과를 직접 다룰 수 있도록 predict_faiss_knn_topk를 사용합니다.

# Task2 Accuracy (Top-K Same-Class)를 계산하는 함수
def task2_score(y_true, topk_preds, topk=10):
    match_counts = []
    for true_label, pred_list_np in zip(y_true, topk_preds):
        # NumPy 배열을 Python 리스트로 변환하여 .count() 메서드 사용
        pred_list = pred_list_np.tolist()
        # topk_preds의 각 요소(pred_list)는 실제 라벨이 포함된 횟수 / topk
        # 예를 들어, topk=10일 때, 'car' 라벨이 3번 등장하면 3/10 = 0.3
        match_counts.append(pred_list.count(true_label) / topk)
    return np.mean(match_counts)

def plot_confusion_matrix(cm, class_names, title='Confusion Matrix', normalize=False):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt=".2f" if normalize else "d", cmap="Blues",
                xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()


✔ 특징 추출 함수 정의 완료


In [3]:
# 🔹 경로 설정
MODEL_DIR = './model_outputs2'
QUERY_IMAGE_DIR = 'C:/Users/bvb09/Downloads/query/query'  # <- 여기에 쿼리 이미지 폴더 넣기

# 🔹 모델 및 도구 불러오기
faiss_index = faiss.read_index(os.path.join(MODEL_DIR, 'faiss_index.idx'))
le = joblib.load(os.path.join(MODEL_DIR, 'label_encoder.pkl'))
pca_hog = joblib.load(os.path.join(MODEL_DIR, 'pca_hog.pkl'))
pca_sift = joblib.load(os.path.join(MODEL_DIR, 'pca_sift.pkl'))
bovw_vocabulary = np.load(os.path.join(MODEL_DIR, 'bovw_vocabulary.npy'))
train_labels_for_pred = joblib.load(os.path.join(MODEL_DIR, 'labels.pkl'))

In [4]:
# ================================
# 1. 쿼리 이미지 로드 (라벨 없음)
# ================================
def load_images_nolabel(folder_path, resize_to=(120, 120)):
    """
    폴더 내 모든 이미지를 로드하고 (옵션) 리사이즈해서 리스트로 반환
    """
    images, image_paths = [], []
    valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')

    for fname in os.listdir(folder_path):
        if fname.lower().endswith(valid_exts):
            img_path = os.path.join(folder_path, fname)
            img = cv2.imread(img_path)
            if img is not None:
                if resize_to is not None:
                    img = cv2.resize(img, resize_to, interpolation=cv2.INTER_AREA)
                images.append(img)
                image_paths.append(img_path)
            else:
                print(f"⚠️ 이미지 로드 실패: {img_path}")

    return images, image_paths


In [5]:
# 🔹 이미지 로딩
images, image_paths = load_images_nolabel(QUERY_IMAGE_DIR)
print(f"✔ 쿼리 이미지 {len(images)}장 로드 완료.")

✔ 쿼리 이미지 100장 로드 완료.


In [6]:
features_color = np.array([extract_color_histogram_features(img) for img in tqdm(images, desc="Color")])
features_laws = np.array([extract_laws_energy_features(img) for img in tqdm(images, desc="LAWS")])
hog_list = [extract_hog_features(img) for img in tqdm(images, desc="HOG")]
features_hog = pca_hog.transform(np.vstack(hog_list))

sift_des_list = [extract_sift_descriptors_from_array(img) for img in tqdm(images, desc="SIFT")]
sift_des_list = [
    pca_sift.transform(d) if d is not None and d.shape[1] == 128 else None
    for d in sift_des_list
]

valid_des = [d for d in sift_des_list if d is not None]
bovw_hist_partial = parallel_create_bovw_histograms(valid_des, bovw_vocabulary, n_jobs=-1)

bovw_hist = []
idx = 0
for d in sift_des_list:
    if d is not None:
        bovw_hist.append(bovw_hist_partial[idx])
        idx += 1
    else:
        bovw_hist.append(np.zeros(bovw_vocabulary.shape[0]))
bovw_hist = np.array(bovw_hist)

SIFT: 100%|██████████| 100/100 [00:00<00:00, 321.93it/s]


In [7]:
# ================================
# 4. 특징 벡터 결합 및 예측
# ================================
X_combined = combine_features(features_color, features_laws, features_hog, bovw_hist).astype(np.float32)

# 🔹 Faiss로 Top-K 예측 수행
topk = 10
D, I, predicted_neighbor_labels_topk = predict_faiss_knn_topk(
    faiss_index, train_labels_for_pred, X_combined, k=topk
)

# 🔹 Top-1 예측 결과 (가장 가까운 이웃의 라벨)
predicted_labels_encoded_top1 = predicted_neighbor_labels_topk[:, 0]

print(f"\n✔ Top-{topk} 예측 완료: {predicted_neighbor_labels_topk.shape}")

# ================================
# 5. 평가 지표 출력
# ================================
# 🔹 Top-1 예측 라벨 (문자열)
predicted_labels_top1 = le.inverse_transform(predicted_labels_encoded_top1)

# 🔹 이미지 파일명만 추출
image_names = [os.path.basename(path) for path in image_paths]

# 🔹 결과 DataFrame
results_simple_df = pd.DataFrame({
    'image': image_names,
    'predicted_label': predicted_labels_top1
})

# 🔹 저장
csv_path = os.path.join(MODEL_DIR, 'c1_t1_a2.csv')
results_simple_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f"✅ Top-1 예측 결과 저장 완료: {csv_path}")

# 🔹 확인
results_simple_df.head()

# 🔹 라벨 문자열 변환
topk_labels_str = [le.inverse_transform(row) for row in predicted_neighbor_labels_topk]

# 🔹 파일 이름만 추출
image_names = [os.path.basename(p) for p in image_paths]

# 🔹 DataFrame 생성
topk_df = pd.DataFrame(topk_labels_str)
topk_df.insert(0, 'image', image_names)  # 첫 열에 이미지 이름 추가

# 🔹 저장
csv_path = os.path.join(MODEL_DIR, 'c1_t2_a2.csv')
topk_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f"✅ Top-10 예측 결과 CSV 저장 완료: {csv_path}")


✔ Top-10 예측 완료: (100, 10)
✅ Top-1 예측 결과 저장 완료: ./model_outputs2\c1_t1_a2.csv
✅ Top-10 예측 결과 CSV 저장 완료: ./model_outputs2\c1_t2_a2.csv
